In [6]:
%pip install dataretrieval --break-system-packages
%pip install panda --break-system-packages
%pip install numpy --break-system-packages

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [7]:
from dataretrieval import nwis
from dataretrieval import wqp

In [8]:
# Choptank River near Greensboro, MD
siteNumber = '01491000'
chop_tank_info, md = nwis.get_info(sites=siteNumber)
parameterCd = '00060'

# raw daily data
rawDailyData, md = nwis.get_dv(sites=siteNumber, parameterCd=parameterCd, start="1980-01-01", end="2010-01-01")

# sample data Nitrate:
parameterCd = "00618"
qwData, md = nwis.get_qwdata(sites=siteNumber, parameterCd=parameterCd, start="1980-01-01", end="2010-01-01")

pCode, md = nwis.get_pmcodes(parameterCd=parameterCd)

/home/vscode/.local/lib/python3.11/site-packages/dataretrieval/nwis.py:193: UserWarning: WARNING: Starting in March 2024, the NWIS qw data endpoint is retiring and no longer receives updates. For more information, refer to https://waterdata.usgs.gov.nwis/qwdata and https://doi-usgs.github.io/dataRetrieval/articles/Status.html or email CompTools@usgs.gov.
  warnings.warn(


KeyError: 'sample_start_time_datum_cd'

In [9]:
siteNumbers = ["01491000","01645000"]
siteINFO, md = nwis.get_iv(sites=siteNumbers)

In [10]:
dailyDataAvailable, md = nwis.get_dv(sites=siteNumbers, statCd="00003")